In [1]:
import pandas as pd
import numpy as np
data = pd.read_excel('./Problem_C_Data_Wordle.xlsx')
print(data)



     Unnamed: 0           Unnamed: 1      Unnamed: 2 Unnamed: 3  \
0           NaN                 Date  Contest number       Word   
1           NaN  2022-12-31 00:00:00             560      manly   
2           NaN  2022-12-30 00:00:00             559      molar   
3           NaN  2022-12-29 00:00:00             558      havoc   
4           NaN  2022-12-28 00:00:00             557      impel   
..          ...                  ...             ...        ...   
355         NaN  2022-01-11 00:00:00             206      drink   
356         NaN  2022-01-10 00:00:00             205      query   
357         NaN  2022-01-09 00:00:00             204      gorge   
358         NaN  2022-01-08 00:00:00             203      crank   
359         NaN  2022-01-07 00:00:00             202      slump   

                      Unnamed: 4           Unnamed: 5 Percent in  Unnamed: 7  \
0    Number of  reported results  Number in hard mode       1 try    2 tries   
1                          20380   

In [2]:
data = data.iloc[1:,4]
print(data)
data = np.array(data)
print(data)

1       20380
2       21204
3       20001
4       20160
5       20879
        ...  
355    153880
356    107134
357     91477
358    101503
359     80630
Name: Unnamed: 4, Length: 359, dtype: object
[20380 21204 20001 20160 20879 20011 15554 20281 21937 20490 22180 24137
 26010 22166 22336 22853 22176 20824 24101 22873 21947 21157 23640 21199
 24899 23509 23153 25577 23873 24646 22628 2569 23739 26051 25206 26381
 24197 27705 26663 27437 24288 24991 24749 29208 27465 25576 27475 26536
 25085 24660 25993 27467 28984 27213 26096 31068 29743 27330 29554 27670
 27502 26498 24672 25156 27905 27609 30063 28953 28947 29279 29084 28637
 28741 28322 28612 31269 30459 30403 28906 27197 29151 28575 26878 28408
 26905 29026 32522 30935 32014 32288 30088 28202 31223 30477 31355 30985
 31706 28994 32777 31509 34455 31976 31277 35050 33102 33418 37309 33344
 32142 29497 29147 27887 29237 32172 31962 30992 32734 32733 32018 31191
 35724 31903 35343 33660 34281 30214 31241 34716 36737 33700 33549 35888

+ 使用60个过去的值预测预测60天后的值 比如1，60个数据 预测第120天的 因为题目的12月31号到3月1号刚好是60天
+ 所以窗口大小为60

In [3]:
class Dataset:
    def __init__(self,data,valid_rate=0.2,win_size=60,future=1,pred_win=10) -> None:
        self.data = data
        self.valid_rate = valid_rate
        self.win_size = win_size
        self.future = future
        self.pred_win = pred_win
        self.data_num =len(self.data)
    def split_data(self,x,y):
        x_train,x_valid = x[:round(self.data_num*self.valid_rate)],x[round(self.data_num*self.valid_rate):]
        y_train,y_valid = y[:round(self.data_num*self.valid_rate)],y[round(self.data_num*self.valid_rate):]
        return x_train,y_train,x_valid,y_valid
    def normalize(self,data):
        X=[]
        Y=[]
        for data_x,data_y in data:
            x = [xx/data_x[0]-1 for xx in data_x]
            y = [yy/data_x[0]-1 for yy in data_y]
            
            X.append(x)
            Y.append(y)
        return np.array(X),np.array(Y)
    def fnormalize(self,x,y,x0):
        x = [(xx+1)*x0 for xx in x]
        y = [(yy+1)*x0 for yy in y]
        return x, y
    def get_data(self):

        x=[]
        y=[]
        for i in range(self.win_size,self.data_num-self.future-self.pred_win+2):
            x.append(self.data[i-self.win_size:i])
            y.append(self.data[i+self.future-1:i+self.future-1+self.pred_win])
        x = np.array(x)
        y = np.array(y)
        x,y = self.normalize(zip(x,y))
        return self.split_data(x,y)
        # return x, y
        

In [4]:
lstm_data = Dataset(
    data=data,
    
)

In [5]:
x_train,y_train,x_valid,y_valid = lstm_data.get_data()
print(x_train)

print(y_train)
# x,y = lstm_data.get_data()
# print(x,y)
# print(x.shape,y.shape)

[[ 0.          0.0404318  -0.01859666 ...  0.34102061  0.4501472
   0.35770363]
 [ 0.         -0.05673458 -0.04923599 ...  0.39379362  0.30494246
   0.29701943]
 [ 0.          0.0079496   0.04389781 ...  0.38343083  0.37503125
   0.32483376]
 ...
 [ 0.         -0.00666006 -0.02192698 ...  0.06701049  0.18569623
   0.25472181]
 [ 0.         -0.01536928 -0.01179343 ...  0.19364599  0.26313437
   0.15871269]
 [ 0.          0.00363167 -0.01099976 ...  0.28285086  0.17679925
   0.17152635]]
[[ 3.49460255e-01  3.00196271e-01  2.10598626e-01  2.34347399e-01
   3.69234544e-01  3.54710500e-01  4.75122669e-01  4.20657507e-01
   4.20363101e-01  4.36653582e-01]
 [ 2.49669874e-01  1.63554046e-01  1.86379928e-01  3.16025278e-01
   3.02065648e-01  4.17798529e-01  3.65449915e-01  3.65166950e-01
   3.80824373e-01  3.71627995e-01]
 [ 2.33538323e-01  2.57737113e-01  3.95180241e-01  3.80380981e-01
   5.03074846e-01  4.47577621e-01  4.47277636e-01  4.63876806e-01
   4.54127294e-01  4.31778411e-01]
 [ 2.478

In [1]:
import tensorflow as tf
from tensorflow import keras

class model:
    def __init__(self):
        pass
    def model(self):
        pass